In [12]:
language = 'karelian'
real_name = 'Karjalan'
gpt_name = 'figpt' #rugpt
translation_word = 'Käännös' # Перевод
#Karjalan Lyydin
# вепсский людиковский карельский селькупский кетский чукотский эвенкийский

# Dataset Creation

In [3]:
with open(f'low_resource_mt/datasets/{language}/src-train.txt') as srctrain, open(f'low_resource_mt/datasets/{language}/tgt-train.txt') as tgttrain:
    srctrain = srctrain.readlines()
    tgttrain = tgttrain.readlines()
    train = [(x.strip(), y.strip()) for x,y in zip(srctrain, tgttrain)]

with open(f'{gpt_name}_{language}_train.txt', 'w') as file:
    for line in train:
        prompt = f'{real_name.title()}: {line[0].strip()}. {translation_word}: {line[1].strip()} <|endoftext|>'
        file.write(prompt+'\n')
    
            

In [4]:
with open(f'low_resource_mt/datasets/{language}/src-val.txt') as srcval, open(f'low_resource_mt/datasets/{language}/tgt-val.txt') as tgtval:
    srcval = srcval.readlines()
    tgtval = tgtval.readlines()
    val = [(x.strip(), y.strip()) for x,y in zip(srcval, tgtval)]

with open(f'{gpt_name}_{language}_val.txt', 'w') as file2:
    for line in val:
        prompt = f'{real_name.title()}: {line[0].strip()}. {translation_word}:'
        file2.write(prompt+'\n')
    
            

# Training

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
--output_dir=rugpt_models/weights \
--model_type=gpt2 \
--line_by_line \
--model_name_or_path=sberbank-ai/rugpt3large_based_on_gpt2 \
--do_train \
--train_data_file=rugpt_yeps_train.txt \
--do_eval \
--eval_data_file=rugpt_yeps_val.txt \
--per_gpu_train_batch_size 1 \
--per_gpu_eval_batch_size 1 \
--gradient_accumulation_steps 1 \
--num_train_epochs 5 \
--save_total_limit 1 \
--block_size 1024 \
--overwrite_output_di

In [ ]:
CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
--output_dir=yeps_models/weights \
--model_type=gpt2 \
--line_by_line \
--model_name_or_path=Finnish-NLP/gpt2-large-finnish \
--do_train \
--train_data_file=figpt_yeps_train.txt \
--do_eval \
--eval_data_file=figpt_yeps_val.txt \
--per_gpu_train_batch_size 1 \
--per_gpu_eval_batch_size 1 \
--gradient_accumulation_steps 1 \
--num_train_epochs 5 \
--save_total_limit 1 \
--block_size 1024 \
--overwrite_output_di

# Evaluation

In [1]:
from transformers import file_utils
print(file_utils.default_cache_path)

/home/anmosolova/.cache/huggingface/transformers


In [2]:
import warnings
warnings.filterwarnings("ignore")
import transformers
transformers.logging.set_verbosity_error()

In [17]:
with open(f'low_resource_mt/datasets/{language}/src-test.txt') as srctest, open(f'low_resource_mt/datasets/{language}/tgt-test.txt') as tgttest:
    srctest = srctest.readlines()
    tgttest = tgttest.readlines()
    test = [(x.strip(), y.strip()) for x,y in zip(srctest, tgttest)]

testds = []
#with open('rugpt_evenki_val.txt', 'w') as file2:
for line in test:
    prompt = f'{real_name.title()}: {line[0].strip()}. {translation_word}:'
    testds.append(prompt)

In [6]:
import numpy as np
import torch

np.random.seed(42)
torch.manual_seed(42)

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tok = GPT2Tokenizer.from_pretrained("finnish_models/weights") #загрузка дообученного токенизатора 

model = GPT2LMHeadModel.from_pretrained("finnish_models/weights") #загрузка дообученной модели

model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [25]:
inpt = tok.encode(testds[0], return_tensors="pt")
out = model.generate(inpt.cuda(), max_length=130)
trans = tok.decode([x.item() if x != 50257 else 0 for x in out[0]])
trans

'Karjalan: показывает hyvät šomat. Käännös: красивает были <|endoftext|>'

In [5]:
max([len(x) for x in testds])

219

In [22]:
len(testds)

97

In [7]:
preds = []
for num, line in enumerate(testds):
    inpt = tok.encode(line, return_tensors="pt")
    out = model.generate(inpt.cuda(), max_length=200)
    trans = tok.decode([x.item() if x != 50257 else 0 for x in out[0]])
    preds.append(trans)
    #print(trans)
    if num % 20 == 0:
        print(num)

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [27]:
with open('karelian_fingpt_large_results.txt', 'w') as file:
    for line in preds:
        file.write(line+'\n')

In [18]:
from compute_bleu import compute_4_bleu
hyps = [x.split('Käännös:')[1].split('<|endoftext|>')[0].strip() for x in preds]
refs = tgttest
hyps = [hyp.strip().split() for hyp in hyps]
refs = [[ref.strip().split()] for ref in refs]
result = compute_4_bleu(refs, hyps)

BLEU1 = 11.7/100
BLEU2 = 3.8/100
BLEU3 = 0.0/100
BLEU4 = 0.0/100


/home/anmosolova/mtenv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [10]:
preds[:10]

['Vepsän: heinreged olibad pitkad. Käännös: серпили были длинная письмо <|endoftext|>',
 'Vepsän: a siga maa do kaik träsiše. Käännös: а там подошел все трясет <|endoftext|>',
 'Vepsän: götihe nägeb končaišei. Käännös: повесились видит кончились <|endoftext|>',
 'Vepsän: žaruaigou vertes palotes hiinäntehtes jüibä jouhveden. Käännös: жаркое в расчете на воскресенье муки помоги муки в полотно <|endoftext|>',
 'Vepsän: painįi ubehed kunutal. Käännös: ступника напокнулась курницу <|endoftext|>',
 'Vepsän: kacun tagaze ei vei nütkäitase. Käännös: за горю не вытащится теперь <|endoftext|>',
 'Vepsän: händikhid rikliba. Käännös: волков убили <|endoftext|>',
 'Vepsän: eli sid ii edahan bajar kulišti necen radon prihäine ninga bohattui. Käännös: жил он не дал барин барин этот барин бабаяга парня богатов <|endoftext|>',
 'Vepsän: hän ümbri penzast jänišse mihail jälghe jänišad vuu sid koir töndui. Käännös: он вокруг когда медведь зайца еще заяца появился заяц зайца зайца <|endoftext|>',
 'Vepsä

In [12]:
refs[:10]

[[['думыто', 'еще', 'молодые', 'были']],
 [['какие', 'семьи']],
 [['он',
   'женат',
   'был',
   'в',
   'сигнаволоке',
   'в',
   'семье',
   'митрюой',
   'такая',
   'была',
   'рослая',
   'баба',
   'жену',
   'только',
   'не',
   'помню',
   'как',
   'ее',
   'звали']],
 [['были',
   'луга',
   'и',
   'поближе',
   'в',
   'конце',
   'пелдожи',
   'в',
   'сторону',
   'лужмы',
   'за',
   'своим',
   'озером']],
 [['детей',
   'мало',
   'было',
   'сын',
   'да',
   'дочь',
   'и',
   'у',
   'мийккула',
   'были',
   'дочь',
   'и',
   'сын']],
 [['ох', 'вы', 'черти', 'где', 'ходите']],
 [['в',
   'киннерме',
   'была',
   'церковь',
   'святого',
   'пантелея',
   'святому',
   'пантелею',
   'молиться',
   'ходили',
   'кто',
   'хочет',
   'так',
   'идет',
   'а',
   'кто',
   'не',
   'хочет',
   'не',
   'идет']],
 [['ну',
   'а',
   'потом',
   'видишь',
   'ли',
   'федя',
   'во',
   'время',
   'войны',
   'умер']],
 [['ну', 'а', 'в', 'других', 'деревнях', 'кото